In [1]:
import xgboost as xgb
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score

from sklearn import metrics

### 1) Data Preparing

In [2]:
score = pd.read_csv("C:/Users/jiho0/Desktop/Data_Analysis/MACHINE_LEARNING/data/scores.csv", index_col = 0)
score.head(5)

,school,sex,age,address,famsize,Pstatus,Medu,Fedu,Mjob,Fjob,...,famrel,freetime,goout,Dalc,Walc,health,absences,G1,G2,G3
1,GP,F,18,U,GT3,A,4,4,at_home,teacher,...,4,3,4,1,1,3,6,5,6,6
2,GP,F,17,U,GT3,T,1,1,at_home,other,...,5,3,3,1,1,3,4,5,5,6
3,GP,F,15,U,LE3,T,1,1,at_home,other,...,4,3,2,2,3,3,10,7,8,10
4,GP,F,15,U,GT3,T,4,2,health,services,...,3,2,2,1,1,5,2,15,14,15
5,GP,F,16,U,GT3,T,3,3,other,other,...,4,3,2,1,2,5,4,6,10,10


In [3]:
#더미변수화 
score1 = pd.get_dummies(score, drop_first = True, dtype= 'int64')

score1.head(5)

,age,Medu,Fedu,traveltime,studytime,failures,famrel,freetime,goout,Dalc,...,guardian_mother,guardian_other,schoolsup_yes,famsup_yes,paid_yes,activities_yes,nursery_yes,higher_yes,internet_yes,romantic_yes
1,18,4,4,2,2,0,4,3,4,1,...,1,0,1,0,0,0,1,1,0,0
2,17,1,1,1,2,0,5,3,3,1,...,0,0,0,1,0,0,0,1,1,0
3,15,1,1,1,2,3,4,3,2,2,...,1,0,1,0,1,0,1,1,1,0
4,15,4,2,1,3,0,3,2,2,1,...,1,0,0,1,1,1,1,1,1,1
5,16,3,3,1,2,0,4,3,2,1,...,0,0,0,1,1,0,1,1,0,0


In [4]:
#G1 G2 칼럼 제외
score2 = score1.drop(['G1' ,'G2'], axis = 1)

#데이터의 라벨을 분할
x, y = score1.drop(['G3'], axis = 1), score['G3']

#train test 분리
train_data, holdout_data, train_label, holdout_label = train_test_split(x, y, test_size = 0.3,
                                                                       random_state = 1234)
print("train_Data's shape : {0}".format(train_data.shape))
print("train_label's shape : {0}".format(train_label.shape))
print("holdout_data's shape : {0}".format(holdout_data.shape))
print("holdout_label's shape : {0}".format(holdout_label.shape))

#xgb용 데이터 생성
train_set = xgb.DMatrix(data = train_data, label = train_label)
holdout_set = xgb.DMatrix(data = holdout_data, label = holdout_label)

train_Data's shape : (730, 41)
train_label's shape : (730,)
holdout_data's shape : (314, 41)
holdout_label's shape : (314,)


### 2) Bayesian Optimization

In [5]:
#bayesian optimization을 적용
from bayes_opt import BayesianOptimization

In [6]:
#임의의 함수 최적화(돌아가는 방식을 이해)
def sample_function(x,y) : 
    return x + y

pbounds = {'x' : (2,4), 'y' : (-3,3)}

optimizer = BayesianOptimization(
f = sample_function,
pbounds = pbounds,
random_state = 1)

In [7]:
optimizer.maximize(
    init_points = 2,
    n_iter = 3
)

|   iter    |  target   |     x     |     y     |
-------------------------------------------------
|  1        |  4.156    |  2.834    |  1.322    |
|  2        |  0.8142   |  2.0      | -1.186    |
|  3        |  1.431    |  2.218    | -0.7867   |
|  4        |  6.589    |  4.0      |  2.589    |
|  5        |  5.0      |  2.0      |  3.0      |


이와 같이 범위의 시작과 끝 값을 정해주면 알아서 그 최대값을 찾아주는 것 같다.

### 3) Bayesian Optimization -  xgboost

- Bayesian Optimization on XGBoost

https://github.com/fmfn/BayesianOptimization

- acq는 acquition function 인데 UCB(upper confidence bound) / EI(expected improvment) 등이 존재 

- n_iter : How many steps of bayesian optimization you want to perform. The more steps the more likely to find a good maximum you are.

- init_point : How many steps of random exploration you want to perform. Random exploration can help by diversifying the exploration space.

#### a) Acquition Function = EI

In [8]:
#원하는 파라미터의 값들과 범위를 지정해준다.
pbounds = { 'learning_rate': (0.01, 1.0),  
           'n_estimators': (10, 5000),
           'max_depth': (3,10),   
           'min_child_weight' : (3,10),
    'subsample': (0.4, 1.0),  
    'colsample_bytree': (0.4, 1.0),   
    'gamma': (0, 1)}

#우리가 Optimize할 함수 지정
def xgboost_hyper_param(learning_rate, n_estimators, max_depth, min_child_weight,
                        subsample, colsample_bytree, gamma):

    X = train_data
    y = train_label
    
    max_depth = int(max_depth)
    n_estimators = int(n_estimators)
    
    reg_xgb = xgb.XGBRegressor( max_depth=max_depth, 
                        min_child_weight = min_child_weight,
                        learning_rate=learning_rate,
                        n_estimators=n_estimators,
                        subsample=subsample,
                        colsample_bytree = colsample_bytree,
                        gamma=gamma)
    
    #이 return값이 손실함수이기 때문에 이를 최대화(원래는 최소화)하는 방향으로 탐색하게 됨.
    return np.mean(cross_val_score(reg_xgb, X, y, cv=5, scoring='neg_root_mean_squared_error'))


optimizer_ei = BayesianOptimization(f=xgboost_hyper_param, 
                                 pbounds=pbounds, #아마 parameter bounds가 아닌가 싶다.
                                 random_state=1234)

optimizer_ei.maximize(init_points=3, n_iter = 100, acq='ei', xi=0.01)

best_param_ei = optimizer_ei.max

print("best params : {0} /n best score : {1}".format(best_param_ei['params'], best_param_ei['target']))

|   iter    |  target   | colsam... |   gamma   | learni... | max_depth | min_ch... | n_esti... | subsample |
-------------------------------------------------------------------------------------------------------------
|  1        | -2.026    |  0.5149   |  0.6221   |  0.4434   |  8.498    |  8.46     |  1.37e+03 |  0.5659   |
|  2        | -2.199    |  0.8811   |  0.9581   |  0.8772   |  5.505    |  6.507    |  3.42e+03 |  0.8276   |
|  3        | -2.065    |  0.6222   |  0.5612   |  0.5081   |  3.096    |  8.41     |  4.414e+0 |  0.6189   |
|  4        | -2.197    |  0.8916   |  0.6296   |  0.7858   |  3.54     |  9.804    |  4.415e+0 |  0.7783   |
|  5        | -2.276    |  0.8122   |  0.8888   |  0.895    |  3.119    |  6.546    |  4.726e+0 |  0.6704   |
|  6        | -2.339    |  0.4056   |  0.935    |  0.5483   |  9.98     |  5.078    |  3.29e+03 |  0.6871   |
|  7        | -5.252    |  0.5765   |  0.2367   |  0.9286   |  4.339    |  8.997    |  4.974e+0 |  0.4373   |
|  8      

In [18]:
#찾은 파라미터로 모형적합.
xgb_ei_fit = xgb.XGBRegressor(
 learning_rate = best_param_ei['params']['learning_rate'],
 n_estimators = int(best_param_ei['params']['n_estimators']),
 max_depth = int(best_param_ei['params']['max_depth']),
 min_child_weight = best_param_ei['params']['min_child_weight'],
 gamma = best_param_ei['params']['gamma'],
 subsample = best_param_ei['params']['subsample'],
 colsample_bytree = best_param_ei['params']['colsample_bytree'],
#  reg_alpha = gsearch_step5.best_params_['reg_alpha'],
#  reg_lambda = gsearch_step5.best_params_['reg_lambda'],
 seed =1234)

xgb_ei_fit.fit(train_data, train_label, eval_metric = 'rmse')

#cv_mean_error 도출
print("CV_error : {0}".format(np.mean(cross_val_score(xgb_ei_fit,
                                                      X = train_data, y = train_label, cv = 5, scoring = 'neg_root_mean_squared_error'))))

#in sample error 도출
ei_pred_train = xgb_ei_fit.predict(train_data)
print('in_sample_error : {0}'.format(np.sqrt(metrics.mean_squared_error(ei_pred_train, train_label))))
      
#hold_out_error 도출
ei_pred_holdout = xgb_ei_fit.predict(holdout_data)
print('out_of_sample_error : {0}'.format(np.sqrt(metrics.mean_squared_error(ei_pred_holdout, holdout_label))))


CV_error : -1.738480364657637
in_sample_error : 0.2512176523407838
out_of_sample_error : 1.596483822736031


과적합이 발생해버렸지만,,, 이건 시간 관계상 n_iter를 적게(10)으로 줘서 그런다.
주말에 다시 해보면 다른 결과가 나올 것 같다.

#### b) Acquition Function = UCB

In [12]:
#원하는 파라미터의 값들과 범위를 지정해준다.
pbounds = { 'learning_rate': (0.01, 1.0),  
           'n_estimators': (1000, 5000),
           'max_depth': (3,10),   
           'min_child_weight' : (3,10),
    'subsample': (0.4, 1.0),  
    'colsample_bytree': (0.4, 1.0),   
    'gamma': (0, 1)}

#우리가 Optimize할 함수 지정
def xgboost_hyper_param(learning_rate, n_estimators, max_depth, min_child_weight,
                        subsample, colsample_bytree, gamma):

    X = train_data
    y = train_label
    
    max_depth = int(max_depth)
    n_estimators = int(n_estimators)
    
    reg_xgb = xgb.XGBRegressor(max_depth=max_depth, 
                        min_child_weight = min_child_weight,
                        learning_rate=learning_rate,
                        n_estimators=n_estimators,
                        subsample = subsample,
                        colsample_bytree = colsample_bytree,
                        gamma=gamma)
    
    #이 return값이 손실함수이기 때문에 이를 최대화(원래는 최소화)하는 방향으로 탐색하게 됨.
    return np.mean(cross_val_score(reg_xgb, X, y, cv=5, scoring='neg_root_mean_squared_error'))


optimizer_ucb = BayesianOptimization(f=xgboost_hyper_param, 
                                 pbounds=pbounds, #아마 parameter bounds가 아닌가 싶다.
                                 random_state=1234)

optimizer_ucb.maximize(init_points=3, n_iter = 100, acq='ei', xi=0.01) 

best_param_ucb = optimizer_ucb.max

print("best params : {0} /n best score : {1}".format(best_param_ucb['params'], best_param_ucb['target']))

|   iter    |  target   | colsam... |   gamma   | learni... | max_depth | min_ch... | n_esti... | subsample |
-------------------------------------------------------------------------------------------------------------
|  1        | -2.031    |  0.5149   |  0.6221   |  0.4434   |  8.498    |  8.46     |  2.09e+03 |  0.5659   |
|  2        | -2.2      |  0.8811   |  0.9581   |  0.8772   |  5.505    |  6.507    |  3.734e+0 |  0.8276   |
|  3        | -2.063    |  0.6222   |  0.5612   |  0.5081   |  3.096    |  8.41     |  4.531e+0 |  0.6189   |
|  4        | -2.196    |  0.8916   |  0.6296   |  0.7858   |  3.54     |  9.804    |  4.531e+0 |  0.7783   |
|  5        | -2.277    |  0.8122   |  0.8888   |  0.895    |  3.119    |  6.546    |  4.78e+03 |  0.6704   |
|  6        | -2.339    |  0.4056   |  0.935    |  0.5483   |  9.98     |  5.078    |  3.63e+03 |  0.6871   |
|  7        | -5.252    |  0.5765   |  0.2367   |  0.9286   |  4.339    |  8.997    |  4.979e+0 |  0.4373   |
|  8      